In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import geopandas as gpd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import time

### Задача:

Прогноз продаж одной из популярных моделей [фичерфонов](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%87%D0%B5%D1%80%D1%84%D0%BE%D0%BD) (на картинке ниже пример похожего устройства) в салонах МегаФона
![](https://39.img.avito.st/640x480/8468720439.jpg)

### Исходные данные:

Датасет содержит следующие поля:

1. `point_id` - Индентификатор салона
2. `lon` - Долгота точки
3. `lat` - Широта точки
4. `target` - Значение таргета, усредненное за несколько месяцев и отнормированное

### Требования к решению и советы:

Ниже приведен список из нескольких важных пунктов, необходимых для решения задания. Выполнение каждого из пунктов влияет на итоговую оценку. Вы можете выполнить каждый из пунктов разными способами, самым лучшим будет считаться вариант, когда всё получение и обработка данных будут реализованы на Питоне (пример: вы можете скачать данные из OSM через интерфейс на сайте overpass-turbo или с помощью библиотек `overpass`/`requests`. Оба варианта будут зачтены, но больше баллов можно заработать во втором случае)



1. Салоны расположены в нескольких разных городах, вам необходимо **определить город для каждого салона** (это понадобится во многих частях задания). К этому есть разные подходы. Вы можете провести [обратное геокодирование](https://en.wikipedia.org/wiki/Reverse_geocoding) с помощью геокодера [Nominatim](https://nominatim.org/), доступного через библиотеку `geopy` примерно вот так:
```python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)
```
В таком случае, вам придется обрабатывать полученную строку адреса, чтобы извлечь название города. Также вы можете скачать из OSM или найти в любом другом источнике границы административно территориальных границ России и пересечь с ними датасет с помощью `geopandas.sjoin` (этот вариант более надежный, но нужно будет разобраться с тем, как устроены границы АТД в OSM, обратите внимание на [этот тег](https://wiki.openstreetmap.org/wiki/Key:admin_level))


2. **Используйте данные OSM**: подумайте, какие объекты могут влиять на продажи фичерфонов. Гипотеза: такие телефоны покупают люди, приезжающие в город или страну ненадолго, чтобы вставить туда отдельную симкарту для роуминга. Можно попробовать использовать местоположения железнодорожных вокзалов (изучите [этот тег](https://wiki.openstreetmap.org/wiki/Tag:railway%3Dstation)). Необходимо использовать хотя бы 5 разных типов объектов из OSM. Скорее всего, вам придется качать данные OSM отдельно для разных городов (см. пример для Нью-Йорка из лекции)


3. **Используйте разные способы генерации признаков**: описать положение салона МегаФона относительно станций метро можно разными способами - найти ***расстояние до ближайшей станции***, или же посчитать, сколько станций попадает в ***500 метровую буферную зону*** вокруг салона. Такие признаки будут нести разную информацию. Так же попробуйте поэкспериментировать с размерами буферных зон (представьте, что значат в реальности радиусы 100, 500, 1000 метров). Попробуйте посчитать расстояние до центра города, до других объектов.

4. **Сделайте визуализации**: постройте 2-3 карты для какого нибудь из городов - как распределен в пространстве таргет, где находятся объекты, полученные вами из OSM. Можете использовать любой инструмент - обычный `plot()`, `folium`, `keplergl`. Если выберете Кеплер, обязательно сохраните в файл конфиг карты, чтобы ее можно было воспроизвести. Сделать это можно вот так:

```python
import json
json_data = kepler_map.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)
```
5. Задание не ограничено приведенными выше пунктами, попробуйте нагенерировать интересных признаков, найти в интернете дополнительные данные (в таком случае в комментарии к коду укажите ссылку на ресурс, откуда взяли данные)



6. Это довольно сложная задача - датасет очень маленький, данные по своей природе довольно случайны. Поэтому место и скор на Kaggle не будут играть решающую роль в оценке, но позволят заработать дополнительные баллы

### Read data

In [2]:
train = pd.read_csv('data/mf_geo_train.csv')
test = pd.read_csv('data/mf_geo_test.csv')

In [3]:
from pysal.viz import mapclassify
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon

In [4]:
train.head(2)

,point_id,lon,lat,target
0,ommNZCUV,37.590776,55.848630,-0.348157
1,nMe2LHPb,37.784210,55.750271,1.294206


In [5]:
test.head(2)

,point_id,lon,lat,target
0,F4lXR1cG,37.681242,55.74804,0.0091
1,4LJu4GTf,60.580910,56.79586,0.0091


In [6]:
from geopy.geocoders import Nominatim

In [7]:
import geopy

In [8]:
df_all = pd.concat((train, test)).reset_index(drop=True)

In [9]:
df_all

,point_id,lon,lat,target
0,ommNZCUV,37.590776,55.848630,-0.348157
1,nMe2LHPb,37.784210,55.750271,1.294206
2,ZgodVRqB,39.635721,47.213330,-1.039679
3,0t2jNYdz,37.704570,55.782020,-1.169339
4,U27W4QJ7,37.643983,55.730188,-0.088837
...,...,...,...,...
527,y8oQuX5v,30.353777,60.049792,0.009100
528,4nmfqUw0,92.928927,56.116262,0.009100
529,N9O45mAh,93.015993,56.023697,0.009100
530,h2InCLKa,30.381172,59.871149,0.009100


# Обработка данных

## Определение города

In [10]:
def city(df):
    geolocator = Nominatim(user_agent='name_of_your_app')
    city = [geolocator.reverse((df.lat[i], df.lon[i]), zoom = 16) for i in range(df.shape[0])]
    for i in range(df.shape[0]):
        keys = ['city', 'town', 'county', 'state']
        for key in keys:
            if key in city[i].raw['address'].keys():
                city[i] = city[i].raw['address'][key]
                break
    return city

In [11]:
location = city(df_all)

In [12]:
df_all['location'] = location

In [13]:
df_all.location.value_counts()

Москва                         194
Санкт-Петербург                 93
Самара                          34
Казань                          32
Новосибирск                     29
Екатеринбург                    27
Ростов-на-Дону                  26
Нижний Новгород                 26
Красноярск                      25
Уфа                             24
городской округ Новосибирск      4
Зеленоград                       4
Колпино                          4
Пушкин                           1
Красное Село                     1
Щербинка                         1
Сестрорецк                       1
Кронштадт                        1
Ломоносов                        1
Троицк                           1
Подольск                         1
Парголово                        1
Петергоф                         1
Name: location, dtype: int64

In [14]:
df_all.describe()

,lon,lat,target
count,532.000000,532.000000,532.000000
mean,45.499771,55.990428,0.009137
std,16.801314,2.730798,0.875032
min,29.765540,47.203857,-1.428660
25%,37.486235,55.609990,-0.477818
50%,37.719650,55.787394,0.009100
75%,50.162645,56.353625,0.213704
max,93.023803,60.093366,7.215360


In [15]:
geom = [Point(df_all.lon[i], df_all.lat[i]) for i in range(df_all.shape[0])]
df_all['geometry'] = geom

In [16]:
df_geom = gpd.GeoDataFrame(df_all, crs="EPSG:4326")

## Нахождение bbox для каждого города в списке

In [17]:
import requests

In [18]:
overpass_url = "http://overpass-api.de/api/interpreter"
city_query = """
[out:json];
(relation["place" ~ "town|city"]["addr:country"="RU"];);
out body;
"""

In [19]:
bbox = {}
for city in df_geom.location.unique():
    query = """
    [out:json];
    (relation["name"="{}"]["place" ~ "town|city"]["addr:country"="RU"];);
    out geom;
    """.format(city)
    print(city)
    try:
        bbox[city] = requests.get(overpass_url, params={'data':query}).json()['elements'][0]['bounds'].values()
    except:
        query = """
        [out:json];
        (relation["name"="{}"];);
        out geom;
        """.format(city)
        bbox[city] = requests.get(overpass_url, params={'data':query}).json()['elements'][0]['bounds'].values()

Москва
Ростов-на-Дону
Красноярск
Санкт-Петербург
Уфа
Казань
Екатеринбург
Нижний Новгород
Колпино
Новосибирск
Самара
Красное Село
Сестрорецк
Петергоф
городской округ Новосибирск
Троицк
Зеленоград
Щербинка
Подольск
Пушкин
Кронштадт
Ломоносов
Парголово


In [20]:
bbox_df2 = pd.DataFrame([bbox[city] for city in bbox.keys()], \
                        columns=['minx', 'miny', 'maxx', 'maxy'],\
                        index = [city for city in bbox.keys()]).reset_index()\
                        .rename(columns={"index": "location"})

In [21]:
bbox_df2

,location,minx,miny,maxx,maxy
0,Москва,55.491308,37.290502,55.957772,37.967428
1,Ростов-на-Дону,47.153377,39.404731,47.368733,39.851390
2,Красноярск,55.911840,92.627132,56.133796,93.168561
3,Санкт-Петербург,59.744148,30.043343,60.090737,30.567166
4,Уфа,54.500752,55.778613,54.959875,56.298556
5,Казань,55.603478,48.820585,55.938219,49.381247
6,Екатеринбург,56.593745,60.007081,56.982691,60.943281
7,Нижний Новгород,56.187653,43.727046,56.402380,44.111958
8,Колпино,59.721273,30.506764,59.798711,30.759493
9,Новосибирск,54.800904,82.751132,55.199456,83.160227


## Функция для генерации признаков

### Получение геометрии для разных типов точек по запросу

In [22]:
def get_nodes(url, bboxes, query, retries=5, timeout=15): 
    for i, city in enumerate(bboxes.location.unique()):
        retry = 0
        while retry < retries:
            try:
                response = requests.get(url, \
                                        params={'data': query.format(*bboxes[bboxes.location == city].values[0, 1:])})
                node = response.json()
                print('{} успешно'.format(city))
                break
            except:
                print('При попытке запроса в городе {} произошла ошибка  c кодом {}. \n \
                Таймаут на {} секунд'.format(city, response.status_code, timeout))
                time.sleep(timeout)
                retries += 1
        if len(node['elements']) != 0:
            osm = pd.DataFrame(node['elements'])
            
        if i == 0:
            nodes = osm
        else:
            nodes = pd.concat((nodes, osm))
    try:
        nodes = nodes.join(pd.DataFrame([x for x in nodes['tags']]))\
                .drop('tags', axis = 1).drop_duplicates().reset_index(drop=True)
    except:
        nodes = nodes.values.tolist()
        nodes = pd.DataFrame([x[2] for x in nodes])
        
    nodes['geometry'] = [Point(x, y) for x, y in zip(nodes.lon, nodes.lat)]    
    return gpd.GeoDataFrame(nodes)

### Рассчет ближайшего расстояния

In [23]:
from sklearn.neighbors import KDTree 
from geopy import distance

def find_nearest_distance(nodes, gdf, point):
    tree = KDTree(nodes[['lat', 'lon']], metric='euclidean')
    closest_idx = tree.query([(point.y, point.x)], k=1, return_distance=False)[0]
    nearest_node = nodes.iloc[closest_idx].geometry
    return distance.geodesic((nearest_node.y.values[0], nearest_node.x.values[0]), (point.y, point.x)).km

def calculate_features(nodes, gdf):
    distance = []
    for point in gdf.geometry:
        distance.append(find_nearest_distance(nodes, gdf, point))
    return distance



### Подсчет количества точек входящих в заданный радиус

In [24]:
from functools import partial
import pyproj
from shapely.ops import transform
proj_wgs84 = pyproj.Proj('+proj=latlong +datum=WGS84')

def geodesic_point_buffer(lon, lat, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lon=lon, lat=lat)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf)

In [25]:
def buffer_joins(df, nodes, radius, name='number'):
    buffers = gpd.GeoDataFrame(df['location'], geometry = df['geometry'])
    buffers['buffers'] = buffers.apply(lambda x: \
                                       geodesic_point_buffer(x.geometry.x, x.geometry.y, radius), axis = 1)
    buffers.drop('geometry', axis = 1, inplace = True)
    buffers = buffers.set_geometry('buffers')
    
    nodes = gpd.sjoin(nodes, buffers, op = 'within').groupby(['index_right']).size()
    nodes = gpd.GeoDataFrame(nodes)
    
    return pd.merge(df, nodes, left_index = True, right_index = True, how = 'left').fillna(0)\
            .rename(columns = {0: name})

## Генерация признаков

### Расстояния до Ж/Д станций

In [26]:
railways = """[out:json];
(node["railway"="station"]["station"!="subway"]["subway"!="yes"]({},{},{},{}););
out center;
"""

In [27]:
railway_station = get_nodes(overpass_url, bbox_df2, railways)

Москва успешно
Ростов-на-Дону успешно
Красноярск успешно
Санкт-Петербург успешно
Уфа успешно
Казань успешно
Екатеринбург успешно
Нижний Новгород успешно
Колпино успешно
Новосибирск успешно
Самара успешно
Красное Село успешно
Сестрорецк успешно
Петергоф успешно
городской округ Новосибирск успешно
Троицк успешно
Зеленоград успешно
Щербинка успешно
Подольск успешно
Пушкин успешно
Кронштадт успешно
Ломоносов успешно
Парголово успешно


In [28]:
df_geom['railway_distance'] = calculate_features(railway_station, df_geom)

### Трамваи, метро, монорельсы, станции

In [29]:
subway_query = """[out:json];
(node["railway"~"station|tram|subway|monorail"]({},{},{},{}););
out center;
"""

In [30]:
subway = get_nodes(overpass_url, bbox_df2, subway_query)

Москва успешно
Ростов-на-Дону успешно
Красноярск успешно
Санкт-Петербург успешно
Уфа успешно
При попытке запроса в городе Казань произошла ошибка  c кодом 429. 
                 Таймаут на 15 секунд
Казань успешно
Екатеринбург успешно
Нижний Новгород успешно
Колпино успешно
Новосибирск успешно
Самара успешно
Красное Село успешно
Сестрорецк успешно
Петергоф успешно
городской округ Новосибирск успешно
Троицк успешно
Зеленоград успешно
Щербинка успешно
Подольск успешно
Пушкин успешно
Кронштадт успешно
Ломоносов успешно
Парголово успешно


In [31]:
subway.head(3)

,type,id,lat,lon,alt_name,esr:user,loc_name,loc_name:website,name,name:esr,...,covered,trolleybus,FIXME,check_date:shelter,addr:country,merkator_shift:bing,surveillance,internet_access:fee,internet_access:ssid,geometry
0,node,26999673,55.778834,37.653721,Москва-Октябрьская,060073,Москва-Ленинградская,http://www.tutu.ru/station.php?nnst=79310,Москва-Пассажирская,Москва-Пассажирская,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (37.65372 55.77883)
1,node,285428980,47.229728,39.826995,Москва-Октябрьская,060073,Москва-Ленинградская,http://www.tutu.ru/station.php?nnst=79310,Москва-Пассажирская,Москва-Пассажирская,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (39.82700 47.22973)
2,node,248606294,56.042417,92.763066,Москва-Октябрьская,060073,Москва-Ленинградская,http://www.tutu.ru/station.php?nnst=79310,Москва-Пассажирская,Москва-Пассажирская,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (92.76307 56.04242)


In [32]:
df_geom['transport_distance'] = calculate_features(subway, df_geom)

In [33]:
df_geom = buffer_joins(df_geom, subway, 0.5, name='transport_number')

In [34]:
df_geom.corr()

,lon,lat,target,railway_distance,transport_distance,transport_number
lon,1.000000,-0.250258,-0.026156,0.143445,0.225188,-0.230728
lat,-0.250258,1.000000,-0.125500,-0.127167,-0.217162,0.216361
target,-0.026156,-0.125500,1.000000,-0.018792,0.005714,0.010767
railway_distance,0.143445,-0.127167,-0.018792,1.000000,0.536191,-0.203830
transport_distance,0.225188,-0.217162,0.005714,0.536191,1.000000,-0.322877
transport_number,-0.230728,0.216361,0.010767,-0.203830,-0.322877,1.000000


### Тюрьмы: ближайшее расстояние

В тюрьмах не разрешается пользоваться смартфоном, поэтому логично предположить, что рядом с ними покупают фичерфоны

In [35]:
prison_query = """[out:json];
(relation["amenity"="prison"]({},{},{},{}););
out center;
"""

In [36]:
prisons = get_nodes(overpass_url, bbox_df2, prison_query)

Москва успешно
Ростов-на-Дону успешно
Красноярск успешно
Санкт-Петербург успешно
Уфа успешно
Казань успешно
Екатеринбург успешно
Нижний Новгород успешно
Колпино успешно
Новосибирск успешно
Самара успешно
Красное Село успешно
Сестрорецк успешно
Петергоф успешно
городской округ Новосибирск успешно
Троицк успешно
Зеленоград успешно
Щербинка успешно
Подольск успешно
Пушкин успешно
Кронштадт успешно
Ломоносов успешно
Парголово успешно


In [37]:
prisons.head(3)

,lat,lon,geometry
0,55.955257,37.547494,POINT (37.54749 55.95526)
1,55.761409,37.705680,POINT (37.70568 55.76141)
2,55.791043,37.698394,POINT (37.69839 55.79104)


In [38]:
df_geom['prison_distance'] = calculate_features(prisons, df_geom)

### Военные части

Примерно та же самая мотивация, что и с тюрьмами

In [39]:
military_query = """[out:json];
(relation["landuse"="military"]({},{},{},{}););
out center;
"""

In [40]:
military_bases = get_nodes(overpass_url, bbox_df2, military_query)

Москва успешно
Ростов-на-Дону успешно
Красноярск успешно
Санкт-Петербург успешно
Уфа успешно
Казань успешно
Екатеринбург успешно
Нижний Новгород успешно
Колпино успешно
Новосибирск успешно
Самара успешно
Красное Село успешно
Сестрорецк успешно
Петергоф успешно
городской округ Новосибирск успешно
Троицк успешно
Зеленоград успешно
Щербинка успешно
Подольск успешно
Пушкин успешно
Кронштадт успешно
Ломоносов успешно
Парголово успешно


In [41]:
military_bases.head(3)

,lat,lon,geometry
0,55.789227,37.893811,POINT (37.89381 55.78923)
1,55.901359,37.308018,POINT (37.30802 55.90136)
2,55.897664,37.304957,POINT (37.30496 55.89766)


In [42]:
df_geom['military_distance'] = calculate_features(military_bases, df_geom)

### Магазины техники

Самое очевидное -- магазины с техникой, конкуренция

In [43]:
shop_query = """[out:json];
(node["shop"="electronics"]({},{},{},{}););
out center;
"""

In [44]:
shops = get_nodes(overpass_url, bbox_df2, shop_query)

Москва успешно
Ростов-на-Дону успешно
Красноярск успешно
Санкт-Петербург успешно
Уфа успешно
Казань успешно
Екатеринбург успешно
Нижний Новгород успешно
Колпино успешно
Новосибирск успешно
Самара успешно
Красное Село успешно
Сестрорецк успешно
Петергоф успешно
городской округ Новосибирск успешно
Троицк успешно
Зеленоград успешно
Щербинка успешно
Подольск успешно
Пушкин успешно
Кронштадт успешно
Ломоносов успешно
Парголово успешно


In [45]:
df_geom['shop_distance'] = calculate_features(shops, df_geom)

In [46]:
df_geom = buffer_joins(df_geom, shops, radius = 1, name='shop_number')

In [47]:
df_geom.corr()

,lon,lat,target,railway_distance,transport_distance,transport_number,prison_distance,military_distance,shop_distance,shop_number
lon,1.000000,-0.250258,-0.026156,0.143445,0.225188,-0.230728,0.897270,0.621221,-0.045759,-0.061340
lat,-0.250258,1.000000,-0.125500,-0.127167,-0.217162,0.216361,-0.109860,-0.157128,0.061507,0.150493
target,-0.026156,-0.125500,1.000000,-0.018792,0.005714,0.010767,-0.011626,0.005027,0.018247,0.022275
railway_distance,0.143445,-0.127167,-0.018792,1.000000,0.536191,-0.203830,0.057452,0.133737,-0.089947,-0.084589
transport_distance,0.225188,-0.217162,0.005714,0.536191,1.000000,-0.322877,0.182229,0.252524,-0.042197,-0.114255
transport_number,-0.230728,0.216361,0.010767,-0.203830,-0.322877,1.000000,-0.158321,-0.210873,-0.018182,0.134089
prison_distance,0.897270,-0.109860,-0.011626,0.057452,0.182229,-0.158321,1.000000,0.617907,-0.040606,-0.051878
military_distance,0.621221,-0.157128,0.005027,0.133737,0.252524,-0.210873,0.617907,1.000000,-0.013896,-0.114335
shop_distance,-0.045759,0.061507,0.018247,-0.089947,-0.042197,-0.018182,-0.040606,-0.013896,1.000000,-0.302158
shop_number,-0.061340,0.150493,0.022275,-0.084589,-0.114255,0.134089,-0.051878,-0.114335,-0.302158,1.000000


### Рынки

Как известно, очень много пожилых людей любит ездить по-старинке на рынки, по пути они могут зайти купить телефон

In [48]:
market_query = """[out:json];
(node["amenity"="marketplace"]({},{},{},{}););
out center;
"""

In [49]:
markets = get_nodes(overpass_url, bbox_df2, market_query)

Москва успешно
Ростов-на-Дону успешно
Красноярск успешно
Санкт-Петербург успешно
Уфа успешно
Казань успешно
Екатеринбург успешно
Нижний Новгород успешно
Колпино успешно
Новосибирск успешно
Самара успешно
Красное Село успешно
Сестрорецк успешно
Петергоф успешно
городской округ Новосибирск успешно
Троицк успешно
Зеленоград успешно
Щербинка успешно
Подольск успешно
Пушкин успешно
Кронштадт успешно
Ломоносов успешно
Парголово успешно


In [50]:
markets.head(3)

,type,id,lat,lon,amenity,name,seasonal,source,opening_hours,level,...,image,building,shop,full_name,note,operator,contact:email,landuse,lastcheck,geometry
0,node,517227781,55.895659,37.697186,marketplace,Рынок Садовод,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (37.69719 55.89566)
1,node,638950998,47.267107,39.787587,marketplace,Рынок Садовод,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (39.78759 47.26711)
2,node,1380661866,56.010168,92.845994,marketplace,Рынок Садовод,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (92.84599 56.01017)


In [51]:
df_geom['market_distance'] = calculate_features(markets, df_geom)

In [52]:
df_geom.corr()

,lon,lat,target,railway_distance,transport_distance,transport_number,prison_distance,military_distance,shop_distance,shop_number,market_distance
lon,1.000000,-0.250258,-0.026156,0.143445,0.225188,-0.230728,0.897270,0.621221,-0.045759,-0.061340,0.110453
lat,-0.250258,1.000000,-0.125500,-0.127167,-0.217162,0.216361,-0.109860,-0.157128,0.061507,0.150493,-0.032637
target,-0.026156,-0.125500,1.000000,-0.018792,0.005714,0.010767,-0.011626,0.005027,0.018247,0.022275,0.017744
railway_distance,0.143445,-0.127167,-0.018792,1.000000,0.536191,-0.203830,0.057452,0.133737,-0.089947,-0.084589,0.059164
transport_distance,0.225188,-0.217162,0.005714,0.536191,1.000000,-0.322877,0.182229,0.252524,-0.042197,-0.114255,0.181939
transport_number,-0.230728,0.216361,0.010767,-0.203830,-0.322877,1.000000,-0.158321,-0.210873,-0.018182,0.134089,-0.141654
prison_distance,0.897270,-0.109860,-0.011626,0.057452,0.182229,-0.158321,1.000000,0.617907,-0.040606,-0.051878,0.150937
military_distance,0.621221,-0.157128,0.005027,0.133737,0.252524,-0.210873,0.617907,1.000000,-0.013896,-0.114335,0.274448
shop_distance,-0.045759,0.061507,0.018247,-0.089947,-0.042197,-0.018182,-0.040606,-0.013896,1.000000,-0.302158,0.052702
shop_number,-0.061340,0.150493,0.022275,-0.084589,-0.114255,0.134089,-0.051878,-0.114335,-0.302158,1.000000,-0.062772


### Расстояние до центра города

In [53]:
center_query = """[out:json];
(relation["place" ~ "city"]({},{},{},{}););
out center;
"""

In [54]:
centers = get_nodes(overpass_url, bbox_df2, center_query)

Москва успешно
Ростов-на-Дону успешно
Красноярск успешно
Санкт-Петербург успешно
Уфа успешно
Казань успешно
Екатеринбург успешно
Нижний Новгород успешно
Колпино успешно
Новосибирск успешно
Самара успешно
Красное Село успешно
Сестрорецк успешно
Петергоф успешно
городской округ Новосибирск успешно
Троицк успешно
Зеленоград успешно
Щербинка успешно
Подольск успешно
Пушкин успешно
Кронштадт успешно
Ломоносов успешно
Парголово успешно


In [55]:
df_geom['center_distance'] = calculate_features(centers, df_geom)

## Визуализации для города Москва

In [56]:
import folium
from folium.plugins import HeatMap

In [57]:
lon = (bbox_df2[bbox_df2.location == 'Москва'].minx + bbox_df2[bbox_df2.location == 'Москва'].maxx) / 2
lat = (bbox_df2[bbox_df2.location == 'Москва'].miny + bbox_df2[bbox_df2.location == 'Москва'].maxy) / 2

### Тепловая карта в зависимости от таргета

In [58]:
heat_data = df_geom[:425][df_geom[:425].location == 'Москва'][['lat', 'lon', 'target']].values.tolist()

In [60]:
m = folium.Map(location = [lon, lat], tiles='cartodbpositron')
HeatMap(heat_data, raduis = 0.1).add_to(m)
m

### Исследумые точки в зависиости от таргета

In [61]:
points = pd.concat((df_geom[:425][['point_id', 'location', 'target']], \
                    df_geom[:425].to_crs(3857).buffer(700).to_crs(4326)), axis = 1)
points = points[points.location == 'Москва'].set_geometry(0)

In [62]:
m =  folium.Map(location = [lon, lat], tiles='cartodbpositron')

folium.Choropleth(
    geo_data=points,
    name="target",
    data=points,
    columns = ['point_id', 'target'],
    key_on='feature.properties.point_id',
    fill_color='YlOrRd',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name="Target",
).add_to(m)
m

### Расположение ж/д станций

In [63]:
moscow_bbox = bbox_df2[bbox_df2.location == 'Москва'].values
def moscow_nodes(nodes):
    nodes = nodes[(nodes.lon > moscow_bbox[0, 2]) & \
                    (nodes.lon < moscow_bbox[0, 4]) & \
                   (nodes.lat < moscow_bbox[0, 3]) & \
                   (nodes.lat > moscow_bbox[0, 1])]
    return gpd.GeoDataFrame(nodes, crs="EPSG:4326")

In [64]:
m =  folium.Map(location = [lon, lat], tiles='cartodbpositron')

folium.Choropleth(
    geo_data=moscow_nodes(railway_station),
    fill_opacity=1,
    line_opacity=0.2,
    legend_name="Target",
).add_to(m)
m

### Расположение военных частей

In [65]:
m =  folium.Map(location = [lon, lat], tiles='cartodbpositron')

folium.Choropleth(
    geo_data=moscow_nodes(military_bases),
    fill_opacity=1,
    line_opacity=0.2,
    legend_name="Target",
).add_to(m)
m

### Таргет + Станции

In [66]:
m =  folium.Map(location = [lon, lat], tiles='cartodbpositron')

folium.Choropleth(
    geo_data=moscow_nodes(railway_station),
    fill_opacity=1,
    line_opacity=0.2,
    legend_name="Target",
).add_to(m)

folium.Choropleth(
    geo_data=points,
    name="target",
    data=points,
    columns = ['point_id', 'target'],
    key_on='feature.properties.point_id',
    fill_color='YlOrRd',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name="Target",
).add_to(m)
m

### Fit model

In [67]:
df = pd.DataFrame(df_geom.iloc[:, 3:]).drop('geometry', axis = 1)
df = pd.get_dummies(df, columns = ['location'])

In [68]:
train = df.iloc[:425, :]
test = df.iloc[425:, 1:]

In [69]:
from sklearn.ensemble import RandomForestRegressor

In [70]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('target', axis = 1), train.target)
rf = RandomForestRegressor(max_depth = 4)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4)

In [71]:
mean_absolute_error(y_test, rf.predict(X_test)), mean_absolute_error(y_train, rf.predict(X_train))

(0.6834641909383662, 0.535715254268943)

In [72]:
rf.fit(train.drop('target', axis=1), train[['target']])

RandomForestRegressor(max_depth=4)

### Make submission

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission['target'] = rf.predict(test)
submission.to_csv('data/submission.csv', index=False)